# Annealing algorithm evaluation and OpenJij benchmarking capabilities

Since the annealing algorithm is a heuristic algorithm, it cannot always produce the optimal solution every single time.
Also, since it is a stochastic algorithm, solutions will differ each time. Therefore, when evaluating an algorithm, various kinds of averages are used to evaluate its solution. Commonly used indices include:

- Time to solution (TTS)
- Success probability
- Residual energy

The success probability is the probability that the optimal solution was obtained.
Residual energy is an average value that indicates how close the obtained solution was to the optimal solution.
TTS is the computation time it takes to obtain an optimal solution with a certain probability, and this is often used in various evaluations.

In this note, we descuss the above three evaluation indicators and their measurement methods.

## Time to solution
An annealing algorithm can produce a solution with any computation time.
However, no matter how fast the computation is, it is meaningless if the solution is wrong.
Therefore, we use the computation time until the optimal solution is calculated with the required probability (e.g. the time it takes to obtain the optimal solution with a 90% probability) as an indicator.

As we did in the Introduction to OpenJij note, an annealing algorithm searches for the optimal solution among multiple runs, so it is necessary to take those multiple runs into account when evaluating the computation time.

> For example, even if the probability of finding an optimal solution with a short computation time $\tau_{short}$ is low, annealing multiple times with that computation time $\tau_{short}$ may require less computation time than performing one time with a longer computation time $\tau_{long}$.
> Therefore, when evaluating the computation time, it may not be sufficient to simply compare annealing times.

The TTS is defined as the amount of time required for the above calculation, taking into account the fact that annealing is performed multiple times.
The TTS can be derived as follows.

Let $p_s(\tau)$ be the probability that the optimal solution is calculated in one annealing time $\tau$.
This $p_s(\tau)$ is the success probability used to evaluate the algorithm.
From these definitions, the failure probability that the optimal solution is not calculated in one annealing is:

$$1-p_s(\tau)$$

Let us repeat this $R$ times. Then, the probability that the optimal solution is not calculated in all these $R$ times is:

$$\{ 1-p_s(\tau)\}^R$$

Therefore, $p_R$, the probability of obtaining an optimal solution at least once out of $R$ times can be obtained as follows:

$$p_R = 1-\{ 1-p_s(\tau)\}^R$$

Solving this equation for $R$ yields:

$$R = \frac{\ln(1-p_R)}{\ln\{1-p_s(\tau)\}}$$

Multipling this by a single computation time $\tau$ leads to the total computation time. This is the TTS, the computation time required to obtain a solution.

$${\rm TTS}(\tau, p_R) = \tau R = \tau \frac{\ln(1-p_R)}{\ln\{1-p_s(\tau)\}}$$

${\rm TTS}(\tau, p_R)$ is the total computation time, which takes into account the number of iterations until an optimal solution is obtained with probability $p_R$, when one annealing takes $\tau$ computation time and an optimal solution is obtained with probability $p_s(\tau)$.

$p_R$ is given as a constant when evaluating actual problems.
$p_R=0.99$ is often used for research and other purposes.
$p_s(\tau)$ are calculated at various annealing times $\tau$, and then ${\rm TTS}(\tau, p_R)$ is derived.

In addition, in real problems, there are many cases where the optimal solution is not known.
Therefore, the **Time to Feasible Solution** is also used, calculated using the probability of obtaining a feasible solution instead of the probability of obtaining an optimal solution.

## Success Probability
The success probability simply calculates the probability of obtaining the optimal solution.
Assuming that the optimal solution is obtained $n$ times when annealing is repeated $R$ times, the success probability $p_s$ is:

$$p_s = \frac{n}{R}$$

## Residual energy
Instead of time, as in TTS, an **approximation ratio** is sometimes used, which is a metric that expresses how much cost was reduced compared to the optimal solution.
The approximate ratio is calculated as follows.

$$
r = \langle E \rangle / E_{\min}
$$

In physics, **residual energy**, the difference between the average energy and the optimal value, is also sometimes used.

$$
E_{\text{res}} = \langle E \rangle - E_{\min}
$$
In physics, we often deal with problems where the cost value is proportional to the number of variables $N$, so we can normalize by size like $(\langle E \rangle - E_{\min})/N$ or
Sometimes it is normalized by the optimal value $E_{\min}$, as in $(\langle E \rangle - E_{\min})/|E_{\min}|$.

Since the annealing algorithm is an algorithm for which there exists an asymptotic convergence to the optimal solution, in most cases this residual energy becomes smaller as the annealing time is increased.